In [8]:
# data load
import pandas as pd
import numpy as np

df= pd.read_csv('C:/Users/whgud/TIL/team project/clustered_total.csv')

## 데이터 요약
> - cluster a : 0이면 단기, 1이면 장기
> - cluster bs : a가 0인 데이터(단기) 중에서 안정형 분류. 0, 1일 경우 안정
> - cluster bl : a가 1인 데이터(장기) 중에서 안정형 분류. 0, 1일 경우 안정

> 주의할 점 : cluster bs, bl이 2라고 해서 모두 공격형 투자로 추천할 만한 곡은 아니다.
> 추가적인 기준이 필요하다. (예측 모델 등 활용해 예상 수익이 높게 나온다든지)


### 추출 방법
- cluster a를 통해 장/단기를 나누고, 각각 cluster bs(단기), cluster bl(장기)를 통해 안정적 투자에 추천할 만한 음원들을 고릅니다. (cluster가 0 또는 1인 것들)
- 이후 cluster b가 2인 데이터들 중에서, 거래량(traded), artist power, youtube 등의 피처를 고려하거나 예측 모델을 활용해 공격형 투자에 적합한 것을 골라 보여줍니다.
- cluster a를 통해 둘로 나눈 데이터 각각에서 cluster bs와 bl을 다시 나눴기 때문에, 두 개 컬럼에는 NaN값이 있으며, 서로 중복되지 않습니다. 또한 두 컬럼 모두 cluster 0, 1이 '안정'에 해당하므로 필요에 따라 bs와 bl을 합쳐서 cluster b로 만들어도 무방합니다.

In [17]:
df

,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
0,0,롤린 (Rollin'),브레이브걸스,860000,0.031,0.016,3490,27044,"1/4,782",Rollin',...,2179034,69797587,91793,0,0,4.838218e+06,33.797386,1,NaN,0.0
1,1,미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ),아이유,845000,0.003,-0.004,2219,2350,"1/13,120",REAL,...,2069778,23700238,1139227,76,8,1.042961e+06,3.389527,1,NaN,1.0
2,2,축가,전우성 (노을),550100,0.032,-0.045,7348,17608,"1/2,000","2018戀歌(연가), 6월의 고백 '축가'",...,573964,13663219,11520,28,1,8.081687e+05,3.586103,1,NaN,1.0
3,3,저녁하늘,에일리(AILEE),418000,0.006,0.000,2389,2313,"1/6,715",Invitation,...,1380968,23514941,116574,76,1,2.516956e+06,0.347222,1,NaN,1.0
4,5,눈물 자국,김종국,350000,0.003,0.000,932,905,1/79,러브 인 메모리 2 - 아빠의 노트,...,48656,220760,347262,6,0,3.300290e+05,0.030769,1,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,921,Rap Star,Dok2,16000,0.041,0.026,775,653,"1/1,662",South Korean Rapstar Mixtape,...,118136,1586218,121753,0,0,1.957255e+06,3.469880,1,NaN,2.0
734,922,L4L (Lookin' For Luv) (Feat. Dok2 & The Quiett),"BOBBY (FEAT. DOK2, THE QUIETT)",15700,0.039,0.006,696,613,"1/2,334",쇼미더머니3 BOBBY vs 올티,...,554489,8000752,821989,28,0,1.747679e+06,8.090759,1,NaN,2.0
735,923,"아퍼 (Feat. Kid Milli, Lil tachi, 김승민, NO:EL, C ...",기리보이,15700,0.039,-0.048,1503,616,"1/32,032",100년제전문대학,...,1373434,24567569,95475,76,0,1.961859e+06,34.885057,1,NaN,1.0
736,924,Machine Gun (Feat. MINO),"KUSH (쿠시), Zion.T",16600,0.032,0.000,731,532,"1/2,042",쇼미더머니 5 Special,...,521303,6100787,130206,28,0,8.363969e+05,7.725926,1,NaN,2.0


In [23]:
# 단기투자만 추출
shortterm_df = df[df['cluster_a']==0]
shortterm_df

,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
23,29,I Can't,2PM,122000,0.003,0.162,633,349,"1/3,548",Hands Up,...,39026,387143,2036208,41,0,3.302572e+05,0.347855,0,2.0,NaN
36,45,I Love U Oh Thank U (Feat. 김태우 Of God),MC몽,99000,0.023,0.117,2326,2271,"1/8,250",Loving You (러빙유),...,176983,1381534,20031,0,1,1.907271e+06,1.469337,0,2.0,NaN
45,58,너와 함께,울랄라세션,89000,0.009,0.068,795,768,"1/1,439",너와 함께,...,40927,508048,16530,28,0,1.456323e+06,0.194719,0,2.0,NaN
52,66,내사람 (Ivy Mix),SG 워너비,84700,0.109,0.072,2019,9208,"1/1,900",Ivy Mega Mix 2006 (아이비 메가 믹스 2006),...,2800,14818,56215,76,0,3.037961e+06,250.272727,0,2.0,NaN
111,136,오늘부터 1일 (Day 1),케이윌,58600,0.045,0.102,2840,2613,"1/5,361",ONE FINE DAY,...,1358095,18362043,142767,76,0,3.391033e+06,10.484753,0,1.0,NaN
117,142,BAE,효린,57600,0.040,0.152,2890,2290,"1/1,500",BAE,...,406892,3318199,146346,5,0,2.665110e+06,3.265988,0,2.0,NaN
120,145,Aspirin (아스피린),걸,56800,0.023,0.134,1678,1322,"1/7,892",Rock'n Roll Revolution,...,156158,866058,16471,0,0,1.242433e+05,2.996810,0,2.0,NaN
166,214,Ma Boy (마보이),씨스타19,45900,0.034,0.077,2649,1562,"1/4,444",Ma Boy,...,570107,4294989,5316,76,0,1.982935e+08,19.464286,0,2.0,NaN
172,220,그리워하다,비투비,45200,0.040,0.074,2244,1805,"1/18,851",Brother Act.,...,2316540,67220574,1049617,76,0,2.999906e+06,18.173397,0,0.0,NaN
189,241,Another Day,"먼데이 키즈 (Monday Kiz), 펀치 (Punch)",42800,0.042,0.112,5224,1814,"1/11,161",호텔 델루나 OST Part.1 (tvN 토일드라마),...,1451118,21824075,80478,41,1,1.355180e+06,11.897770,0,1.0,NaN


In [24]:
# 장기투자만 추출
longterm_df = df[df['cluster_a']==1]
longterm_df

,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
0,0,롤린 (Rollin'),브레이브걸스,860000,0.031,0.016,3490,27044,"1/4,782",Rollin',...,2179034,69797587,91793,0,0,4.838218e+06,33.797386,1,NaN,0.0
1,1,미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ),아이유,845000,0.003,-0.004,2219,2350,"1/13,120",REAL,...,2069778,23700238,1139227,76,8,1.042961e+06,3.389527,1,NaN,1.0
2,2,축가,전우성 (노을),550100,0.032,-0.045,7348,17608,"1/2,000","2018戀歌(연가), 6월의 고백 '축가'",...,573964,13663219,11520,28,1,8.081687e+05,3.586103,1,NaN,1.0
3,3,저녁하늘,에일리(AILEE),418000,0.006,0.000,2389,2313,"1/6,715",Invitation,...,1380968,23514941,116574,76,1,2.516956e+06,0.347222,1,NaN,1.0
4,5,눈물 자국,김종국,350000,0.003,0.000,932,905,1/79,러브 인 메모리 2 - 아빠의 노트,...,48656,220760,347262,6,0,3.300290e+05,0.030769,1,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,921,Rap Star,Dok2,16000,0.041,0.026,775,653,"1/1,662",South Korean Rapstar Mixtape,...,118136,1586218,121753,0,0,1.957255e+06,3.469880,1,NaN,2.0
734,922,L4L (Lookin' For Luv) (Feat. Dok2 & The Quiett),"BOBBY (FEAT. DOK2, THE QUIETT)",15700,0.039,0.006,696,613,"1/2,334",쇼미더머니3 BOBBY vs 올티,...,554489,8000752,821989,28,0,1.747679e+06,8.090759,1,NaN,2.0
735,923,"아퍼 (Feat. Kid Milli, Lil tachi, 김승민, NO:EL, C ...",기리보이,15700,0.039,-0.048,1503,616,"1/32,032",100년제전문대학,...,1373434,24567569,95475,76,0,1.961859e+06,34.885057,1,NaN,1.0
736,924,Machine Gun (Feat. MINO),"KUSH (쿠시), Zion.T",16600,0.032,0.000,731,532,"1/2,042",쇼미더머니 5 Special,...,521303,6100787,130206,28,0,8.363969e+05,7.725926,1,NaN,2.0


In [26]:
# 단기 투자 / 안정형 추출
shortterm = df[df['cluster_a']==0]
short_stable = shortterm[(shortterm['cluster_bs']==0)|(shortterm['cluster_bs']==1)] # 0이 강력추천, 1은 보통
short_stable

,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
111,136,오늘부터 1일 (Day 1),케이윌,58600,0.045,0.102,2840,2613,"1/5,361",ONE FINE DAY,...,1358095,18362043,142767,76,0,3.391033e+06,10.484753,0,1.0,NaN
172,220,그리워하다,비투비,45200,0.040,0.074,2244,1805,"1/18,851",Brother Act.,...,2316540,67220574,1049617,76,0,2.999906e+06,18.173397,0,0.0,NaN
189,241,Another Day,"먼데이 키즈 (Monday Kiz), 펀치 (Punch)",42800,0.042,0.112,5224,1814,"1/11,161",호텔 델루나 OST Part.1 (tvN 토일드라마),...,1451118,21824075,80478,41,1,1.355180e+06,11.897770,0,1.0,NaN
266,340,갖고놀래 (Feat. 다이나믹 듀오),범키,36500,0.024,0.352,804,865,"1/7,400",갖고놀래,...,911315,12623358,11374,76,0,6.409088e+05,0.825553,0,1.0,NaN
285,366,니가 뭔데 (Who You?),G-DRAGON,32000,0.041,0.280,1204,1304,"1/12,604",쿠데타 'COUP D'ETAT',...,683276,9542319,854104,12,0,1.596224e+07,18.460280,0,1.0,NaN
413,527,다시 너를,"매드클라운, 김나영",28600,0.018,0.122,596,528,"1/10,206",태양의 후예 OST Part.5 (KBS2 수목드라마),...,1158786,18815139,25879,0,2,4.026821e+06,4.058542,0,1.0,NaN


In [27]:
# 단기 투자 / 공격형 추출
shortterm= df[df['cluster_a']==0]
short_aggresive = shortterm[shortterm['cluster_bs']==2]
short_aggresive

,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
23,29,I Can't,2PM,122000,0.003,0.162,633,349,"1/3,548",Hands Up,...,39026,387143,2036208,41,0,3.302572e+05,0.347855,0,2.0,NaN
36,45,I Love U Oh Thank U (Feat. 김태우 Of God),MC몽,99000,0.023,0.117,2326,2271,"1/8,250",Loving You (러빙유),...,176983,1381534,20031,0,1,1.907271e+06,1.469337,0,2.0,NaN
45,58,너와 함께,울랄라세션,89000,0.009,0.068,795,768,"1/1,439",너와 함께,...,40927,508048,16530,28,0,1.456323e+06,0.194719,0,2.0,NaN
52,66,내사람 (Ivy Mix),SG 워너비,84700,0.109,0.072,2019,9208,"1/1,900",Ivy Mega Mix 2006 (아이비 메가 믹스 2006),...,2800,14818,56215,76,0,3.037961e+06,250.272727,0,2.0,NaN
117,142,BAE,효린,57600,0.040,0.152,2890,2290,"1/1,500",BAE,...,406892,3318199,146346,5,0,2.665110e+06,3.265988,0,2.0,NaN
120,145,Aspirin (아스피린),걸,56800,0.023,0.134,1678,1322,"1/7,892",Rock'n Roll Revolution,...,156158,866058,16471,0,0,1.242433e+05,2.996810,0,2.0,NaN
166,214,Ma Boy (마보이),씨스타19,45900,0.034,0.077,2649,1562,"1/4,444",Ma Boy,...,570107,4294989,5316,76,0,1.982935e+08,19.464286,0,2.0,NaN
241,310,TONIGHT,BIGBANG,38400,0.036,0.123,2464,1380,"1/12,723",빅뱅 미니앨범 4집,...,762041,6188893,2156395,12,0,9.020660e+06,9.451943,0,2.0,NaN
257,328,그리워서 눈물나서,이수영,37100,0.023,0.091,995,850,1/299,유령 OST Part.4 (SBS 월화드라마),...,20753,103283,11037,76,0,3.806210e+05,0.725341,0,2.0,NaN
259,330,일년을 하루같이,바비 킴,37100,0.043,0.085,2293,1580,"1/3,555",쩐의 전쟁 OST 1.5집,...,187965,2000230,9493,0,0,2.445651e+05,21.250996,0,2.0,NaN


In [28]:
# 장기 투자 / 안정형 추출
longterm = df[df['cluster_a']==1]
long_stable = longterm[(longterm['cluster_bl']==0)|(longterm['cluster_bl']==1)] # 0이 강력추천, 1은 보통
long_stable

,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
0,0,롤린 (Rollin'),브레이브걸스,860000,0.031,0.016,3490,27044,"1/4,782",Rollin',...,2179034,69797587,91793,0,0,4.838218e+06,33.797386,1,NaN,0.0
1,1,미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ),아이유,845000,0.003,-0.004,2219,2350,"1/13,120",REAL,...,2069778,23700238,1139227,76,8,1.042961e+06,3.389527,1,NaN,1.0
2,2,축가,전우성 (노을),550100,0.032,-0.045,7348,17608,"1/2,000","2018戀歌(연가), 6월의 고백 '축가'",...,573964,13663219,11520,28,1,8.081687e+05,3.586103,1,NaN,1.0
3,3,저녁하늘,에일리(AILEE),418000,0.006,0.000,2389,2313,"1/6,715",Invitation,...,1380968,23514941,116574,76,1,2.516956e+06,0.347222,1,NaN,1.0
11,12,내입술...따뜻한 커피처럼,샵,229700,0.011,0.000,2308,2452,"1/8,800",Flat Album,...,1034749,12046096,135583,0,0,1.420807e+06,2.040369,1,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,792,BIRD,하성운,20000,0.074,0.000,2920,1473,"1/6,655",My Moment,...,517053,25790547,1466545,0,0,6.243163e+05,10.415214,1,NaN,1.0
653,821,라이딩 (Riding) (Feat. 개코),하성운,19300,0.065,0.000,2522,1264,"1/10,033",라이딩 (Riding),...,325717,11558719,1466545,76,0,2.437195e+05,9.504363,1,NaN,1.0
655,823,Goodbye,웬디 (WENDY),19300,0.056,-0.015,1753,1086,"1/7,477",뷰티 인사이드 OST Part.6 (JTBC 월화드라마),...,968663,15078767,667881,0,0,7.336547e+05,2.382096,1,NaN,1.0
694,867,오.꼭.말,하성운,18000,0.043,0.029,2533,775,"1/5,071",My Moment,...,246563,11832273,1466545,0,0,2.758786e+05,17.789604,1,NaN,1.0


In [32]:
# 장기 투자 / 공격형 예시 
longterm = df[df['cluster_a']==1]
long_aggresive = longterm[longterm['cluster_bl']==2]
long_aggresive

,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
4,5,눈물 자국,김종국,350000,0.003,0.000,932,905,1/79,러브 인 메모리 2 - 아빠의 노트,...,48656,220760,347262,6,0,3.300290e+05,0.030769,1,NaN,2.0
5,6,향수,"케이윌, 전우성 (노을)",330000,0.014,0.000,3926,4468,1/182,이단옆차기 프로젝트 Vol.04,...,263178,1789716,142767,76,1,7.045796e+05,0.363850,1,NaN,2.0
6,7,Fly,SUPER JUNIOR-K.R.Y. (슈퍼주니어-K.R.Y.),305100,0.003,0.000,942,939,1/180,FLY (Digital Single),...,8067,91915,25125,28,0,4.751097e+05,0.052805,1,NaN,2.0
7,8,안녕,박혜경,300600,0.029,0.000,1534,8655,"1/2,706",Serahpim,...,215433,1191758,7446,17,0,1.279667e+06,0.996040,1,NaN,2.0
8,9,하이힐,브레이브걸스,285500,0.019,0.027,1107,5361,"1/1,492",HIGH HEELS,...,385816,4609938,91793,0,0,7.486018e+05,3.652397,1,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,920,ANYMORE,서인영,16000,0.026,0.000,1094,417,"1/1,029",ANYMORE,...,108792,400725,9544,28,0,4.533766e+05,6.219089,1,NaN,2.0
733,921,Rap Star,Dok2,16000,0.041,0.026,775,653,"1/1,662",South Korean Rapstar Mixtape,...,118136,1586218,121753,0,0,1.957255e+06,3.469880,1,NaN,2.0
734,922,L4L (Lookin' For Luv) (Feat. Dok2 & The Quiett),"BOBBY (FEAT. DOK2, THE QUIETT)",15700,0.039,0.006,696,613,"1/2,334",쇼미더머니3 BOBBY vs 올티,...,554489,8000752,821989,28,0,1.747679e+06,8.090759,1,NaN,2.0
736,924,Machine Gun (Feat. MINO),"KUSH (쿠시), Zion.T",16600,0.032,0.000,731,532,"1/2,042",쇼미더머니 5 Special,...,521303,6100787,130206,28,0,8.363969e+05,7.725926,1,NaN,2.0


In [45]:
# 예측을 위한 노래 선택
#  장기 - 안정형 중에서도 cluster_bs값이 0일때 강력추천, 1은 보통
long_st_df=long_stable[long_stable['cluster_bl']==0] # 강력추천만 추출
long_st_df.sort_values(by=['fee_near_year'], axis=0, ascending=False) # 최근 저작권료가 높은 순으로 정렬



,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
0,0,롤린 (Rollin'),브레이브걸스,860000,0.031,0.016,3490,27044,"1/4,782",Rollin',...,2179034,69797587,91793,0,0,4838218.435,33.797386,1,NaN,0.0
317,402,사랑이란 멜로는 없어,전상근,32800,0.161,-0.012,15153,5266,"1/36,000",사랑이란 멜로는 없어,...,2197012,58847259,8751,76,2,2339233.043,918.888889,1,NaN,0.0
129,155,니 소식,송하예,55100,0.055,0.036,8001,3038,"1/21,468",니 소식,...,2477380,80958457,6242,0,2,2449178.542,16.014652,1,NaN,0.0
84,106,선물,멜로망스,67500,0.039,-0.007,2433,2606,"1/60,000",Moonlight,...,3261823,107674989,111816,76,0,2927686.870,299.860000,1,NaN,0.0
47,60,좋니,윤종신,90000,0.021,-0.032,1498,1912,"1/80,000",LISTEN 010 좋니,...,2945106,93872732,105259,76,0,4186446.261,210.169014,1,NaN,0.0
344,432,내 생에 아름다운,케이윌,30600,0.052,-0.078,2590,1576,"1/19,198",뷰티 인사이드 OST Part.4 (JTBC 월화드라마),...,2676336,66045402,142767,0,0,1291036.130,34.865672,1,NaN,0.0
201,258,My Way,이수 (엠씨더맥스),42200,0.033,-0.002,2129,1405,"1/23,997",돈꽃 OST PART.1 (MBC 주말드라마),...,2411550,58000795,183878,5,3,2210557.696,39.567376,1,NaN,0.0
44,56,Beautiful,Crush,90000,0.015,0.000,1387,1338,"1/28,225",도깨비 OST Part 4 (tvN 금토드라마),...,2579311,56774075,212185,47,1,7242039.043,14.102564,1,NaN,0.0
79,99,우연히 봄,"로꼬, 유주 (여자친구)",67700,0.020,0.003,1128,1337,"1/17,170",냄새를 보는 소녀 OST Part.2 (SBS 수목드라마),...,2312973,43851634,176598,76,0,1330191.565,9.972924,1,NaN,0.0
363,462,이 별,길구봉구,30800,0.042,0.000,1811,1304,"1/17,259",Star,...,2385731,58455601,21365,28,0,2601417.348,9.493103,1,NaN,0.0


#### 장기- 안정형의 음원의 경우: 저작권료가 높은 것 중에서도 2020년도 저작권료 대비 가격차이가 가장 적은 것 선정한다. 
- 가격차이가 적은 곡들이 꾸준한 수익률을 가져다 줄거라고 기대되기 때문이다

#### 장기 - 안정형 : 선물(멜로망스), 좋니(윤종신), Beautiful(Crush) 중에 1곡으로 예측모델 학습 예정

In [36]:
# 단기 - 공격형
short_aggresive[short_aggresive['cluster_bs']==2]

,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
23,29,I Can't,2PM,122000,0.003,0.162,633,349,"1/3,548",Hands Up,...,39026,387143,2036208,41,0,3.302572e+05,0.347855,0,2.0,NaN
36,45,I Love U Oh Thank U (Feat. 김태우 Of God),MC몽,99000,0.023,0.117,2326,2271,"1/8,250",Loving You (러빙유),...,176983,1381534,20031,0,1,1.907271e+06,1.469337,0,2.0,NaN
45,58,너와 함께,울랄라세션,89000,0.009,0.068,795,768,"1/1,439",너와 함께,...,40927,508048,16530,28,0,1.456323e+06,0.194719,0,2.0,NaN
52,66,내사람 (Ivy Mix),SG 워너비,84700,0.109,0.072,2019,9208,"1/1,900",Ivy Mega Mix 2006 (아이비 메가 믹스 2006),...,2800,14818,56215,76,0,3.037961e+06,250.272727,0,2.0,NaN
117,142,BAE,효린,57600,0.040,0.152,2890,2290,"1/1,500",BAE,...,406892,3318199,146346,5,0,2.665110e+06,3.265988,0,2.0,NaN
120,145,Aspirin (아스피린),걸,56800,0.023,0.134,1678,1322,"1/7,892",Rock'n Roll Revolution,...,156158,866058,16471,0,0,1.242433e+05,2.996810,0,2.0,NaN
166,214,Ma Boy (마보이),씨스타19,45900,0.034,0.077,2649,1562,"1/4,444",Ma Boy,...,570107,4294989,5316,76,0,1.982935e+08,19.464286,0,2.0,NaN
241,310,TONIGHT,BIGBANG,38400,0.036,0.123,2464,1380,"1/12,723",빅뱅 미니앨범 4집,...,762041,6188893,2156395,12,0,9.020660e+06,9.451943,0,2.0,NaN
257,328,그리워서 눈물나서,이수영,37100,0.023,0.091,995,850,1/299,유령 OST Part.4 (SBS 월화드라마),...,20753,103283,11037,76,0,3.806210e+05,0.725341,0,2.0,NaN
259,330,일년을 하루같이,바비 킴,37100,0.043,0.085,2293,1580,"1/3,555",쩐의 전쟁 OST 1.5집,...,187965,2000230,9493,0,0,2.445651e+05,21.250996,0,2.0,NaN


#### 단기- 공격형의 음원의 경우: 최근 저작권료는 낮지만, 활동이 활발할 것으로 예상되는 가수의 곡으로 선정한다.
- 가수가 컴백시 다시 가격이 상승할 확률이 있고, 거래가가 낮은 곡들을 노린다면, 시세차익을 기대할 수 있기때문이다.

#### 단기 - 공격형 : After Party(김재환), I Can't(2PM) 중에 1곡으로 예측모델 학습 예정

In [44]:
# 단기투자 안정형중에서도 cluster_bs값이 0일때 강력추천, 1은 보통
display(short_stable[short_stable['cluster_bs']==0])
short_stable[short_stable['cluster_bs']==1]

,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
172,220,그리워하다,비투비,45200,0.04,0.074,2244,1805,"1/18,851",Brother Act.,...,2316540,67220574,1049617,76,0,2999905.826,18.173397,0,0.0,NaN


,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
111,136,오늘부터 1일 (Day 1),케이윌,58600,0.045,0.102,2840,2613,"1/5,361",ONE FINE DAY,...,1358095,18362043,142767,76,0,3.391033e+06,10.484753,0,1.0,NaN
189,241,Another Day,"먼데이 키즈 (Monday Kiz), 펀치 (Punch)",42800,0.042,0.112,5224,1814,"1/11,161",호텔 델루나 OST Part.1 (tvN 토일드라마),...,1451118,21824075,80478,41,1,1.355180e+06,11.897770,0,1.0,NaN
266,340,갖고놀래 (Feat. 다이나믹 듀오),범키,36500,0.024,0.352,804,865,"1/7,400",갖고놀래,...,911315,12623358,11374,76,0,6.409088e+05,0.825553,0,1.0,NaN
285,366,니가 뭔데 (Who You?),G-DRAGON,32000,0.041,0.280,1204,1304,"1/12,604",쿠데타 'COUP D'ETAT',...,683276,9542319,854104,12,0,1.596224e+07,18.460280,0,1.0,NaN
413,527,다시 너를,"매드클라운, 김나영",28600,0.018,0.122,596,528,"1/10,206",태양의 후예 OST Part.5 (KBS2 수목드라마),...,1158786,18815139,25879,0,2,4.026821e+06,4.058542,0,1.0,NaN


#### 단기- 안정형의 음원의 경우: 가수의 영향력이 큰 편이라 가격이 급락하지 않을 것으로 예상되는 곡으로 선정한다.
#### 단기 - 안정형 : 그리워하다(비투비), 오늘부터 1일(케이윌), 니가뭔데(GD) 중에 1곡으로 예측모델 학습 예정

In [53]:
# 장기- 공격형 음원을 저작권료가 높은 순으로 정렬 (거래가 대비 저작권료가 높은 곡(저평가된곡)을 찾기 위해)
long_agg_df=long_aggresive[long_aggresive['cluster_bl']==2]
long_agg_df.sort_values(by=['fee_near_year'], axis=0, ascending=False).head(30)


,Unnamed: 0,title,artist,price,fee,day_diff,fee_2020,fee_near_year,fee_share,album,...,listeners,streams,insta_post,pub_power,artist_power,yt_view_mean,traded,cluster_a,cluster_bs,cluster_bl
146,181,minimal warm (취향저격 그녀 X 찬열 (CHANYEOL)),찬열 (CHANYEOL),47000,0.282,-0.115,0,13263,"1/3,400",minimal warm (취향저격 그녀 X 찬열 (CHANYEOL)),...,53201,1003475,2410406,76,0,1.600698e+06,218.500000,1,NaN,2.0
65,82,이별자리,이우,73100,0.147,0.010,0,10731,"1/3,000",이별자리,...,60637,641416,12054,76,0,4.169454e+05,92.025974,1,NaN,2.0
75,93,O Sole Mio,IZ*ONE (아이즈원),69100,0.144,0.000,0,9929,"1/9,001",One-reeler / Act IV,...,74721,1631999,1034759,47,2,4.308000e+05,60.414141,1,NaN,2.0
7,8,안녕,박혜경,300600,0.029,0.000,1534,8655,"1/2,706",Serahpim,...,215433,1191758,7446,17,0,1.279667e+06,0.996040,1,NaN,2.0
136,167,벌써일년,"반하나, MJ (써니사이드)",53800,0.157,-0.002,0,8455,"1/20,000",벌써일년,...,529694,4917998,9305,0,0,2.050590e+05,459.708333,1,NaN,2.0
28,35,내 마음의 사진,송가인,115000,0.061,0.000,11873,6970,"1/3,050",사랑의 불시착 OST Part 6 (tvN 토일 드라마),...,621296,7676168,60432,28,0,4.396424e+05,13.997085,1,NaN,2.0
169,217,그 한마디 (바른연애 길잡이 X 이하이),이하이,44100,0.158,-0.016,0,6965,"1/28,000",그 한마디 (바른연애 길잡이 X 이하이),...,316792,3370075,87537,76,1,5.542398e+05,310.285714,1,NaN,2.0
72,90,모든 밤 너에게 (연애혁명 X 민현 (뉴이스트)),민현 (뉴이스트),69300,0.088,0.044,0,6108,"1/1,991",모든 밤 너에게 (연애혁명 X 민현 (뉴이스트)),...,113273,1519882,913112,0,0,1.439642e+05,54.043478,1,NaN,2.0
56,70,No Woman No Cry,스토니 스컹크,77000,0.079,-0.005,1339,6092,"1/4,203",Skunk Riddim,...,32962,327087,367,12,0,1.539495e+06,7.637232,1,NaN,2.0
8,9,하이힐,브레이브걸스,285500,0.019,0.027,1107,5361,"1/1,492",HIGH HEELS,...,385816,4609938,91793,0,0,7.486018e+05,3.652397,1,NaN,2.0


#### 장기- 공격형의 음원의 경우:  장기 투자를 목표로 하는 사람은 저작권료로 인한 수익 또한 중요하기 때문에 저작권료가 높은 순서 중에서, 거래가 대비 저작권료가 높은 곡으로 선정한다. (저평가된 곡을 찾기 위함)
- 거래가 대비 저작권료가 높다면 저평가되어있는 것이기 때문에 역주행의 가능성이 있어 거래가를 통한 시세차익을 얻을 확률이 있다.
- 단 장기적으로 저작권료를 얻기 위해선 최근에만 반짝 유행하는 곡은 피하는게 좋다.
#### 장기 - 공격형 :두바둡(라붐),Dream Night(ASTRO) 중에 1곡으로 예측모델 학습 예정

### 최종 

- 단기투자에 적합한 곡 : 29개
    - 안정형 : 6개 
    - 공격형 : 23개
- 장기투자에 적합한 곡 : 708개
